In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import ast

from os import listdir
from os.path import isfile, join


In [2]:
# EXTRACT SENTIMENT AGGREGATED BY TIMESPAN
ticker = 'AAPL'
mypath = '/home/simone/Desktop/NLFF/Word2Vec/w2vDataset/'

news =  pd.read_csv(mypath+str(ticker)+'.csv')
news = news.drop(['Unnamed: 0'], axis=1)
news['DATE'] = [datetime.strptime(row, '%Y-%m-%d %H:%M:%S +%f') for row in news['DATE']]
#news['entSentVec'] = [ast.literal_eval(row) for row in news['entSentVec']]
news = news.reset_index(drop=True)



In [3]:


timeSpan = pd.read_csv("/home/simone/Desktop/NLFF/indexes/indexes"+str(ticker)+".csv")
timeSpan = timeSpan['date'].tolist()
# This dataset is already GMT+0
timeSpan = [datetime.strptime(row, '%Y-%m-%d %H:%M:%S') for row in timeSpan]

#print(list(sentiment.columns))

#Prendiamo intervallo massimo dove ci sono entrambi i dataset:
#massimo della data iniziale tra i due
#minimo della data finale tra i due
initDate = max(timeSpan[0], news['DATE'][0])
finalDate = min(timeSpan[len(timeSpan)-1], news['DATE'][len(news)-1])



i = 0
j = 0
# ALLINEAMENTO INIZIO
while(timeSpan[j] < initDate):
    j+=1
init = j
while(news['DATE'][i] <= initDate - timedelta(hours=1)):
    i+=1

print(timeSpan[j])
print(news['DATE'][i])
    
print("Start: "+str(initDate) +"("+str(timeSpan[j])+") i: "+str(i)+" j: "+str(j))
print("End: "+str(finalDate))

vecs = list()
dates = list()
nums = list()
cols = list(news.columns)
cols.remove('DATE')

#ALLINEAMENTO FINE DENTRO I WHILE
while( news['DATE'][i] < finalDate and timeSpan[j] < finalDate ):
    initTime = timeSpan[j]
    normal_sum = np.asarray([0]*300)
    num_tot_news = 0
    while(i<len(news)-1 and timeSpan[j] > news['DATE'][i]):
        if(i%1000 == 0):
            print(i)
        if not (timeSpan[j] > news['DATE'][i] and timeSpan[j-1] <= news['DATE'][i]):
            print("timeSpan["+str(j)+"]: "+str(timeSpan[j])+" news["+str(i)+"] : " +str(news['DATE'][i]) + " timeSpan["+str(j-1)+"]: "+str(timeSpan[j-1]))
            assert False 
        normal_sum = normal_sum + np.asarray(news.loc[i,cols])  
        num_tot_news +=1
        i+=1
    j+=1
    #sentiment.loc[j] = {'initTime':initTime, 'num_news':num_tot_news}
    dates.append(initTime)
    nums.append(num_tot_news)
    if num_tot_news > 0:
        normal_sum = normal_sum / num_tot_news 
    vecs.append(normal_sum)

    
sentiment = pd.DataFrame(data=vecs)
sentiment['initTime'] = dates
sentiment['num_news'] = nums
    


end = j
if(len(sentiment) != end - init):
    print(len(sentiment))
    print(end - init + 1)
    assert False


#sentiment.to_csv('Aggregated_senticnet_dataset/'+str(ticker)+'_senticnet.csv')

2017-04-03 18:00:00
2017-04-03 17:27:34
Start: 2017-04-03 17:27:34(2017-04-03 18:00:00) i: 0 j: 3817
End: 2018-06-21 23:20:00
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000


In [8]:
sentiment = sentiment.sort_values(by=['initTime'])
sentiment = sentiment.reset_index(drop=True)
sentiment.head()
sentiment.isnull().any()

0           False
1           False
2           False
3           False
4           False
5           False
6           False
7           False
8           False
9           False
10          False
11          False
12          False
13          False
14          False
15          False
16          False
17          False
18          False
19          False
20          False
21          False
22          False
23          False
24          False
25          False
26          False
27          False
28          False
29          False
            ...  
272         False
273         False
274         False
275         False
276         False
277         False
278         False
279         False
280         False
281         False
282         False
283         False
284         False
285         False
286         False
287         False
288         False
289         False
290         False
291         False
292         False
293         False
294         False
295         False
296       

In [9]:
sentiment.to_csv('Aggregated_W2V_dataset/'+str(ticker)+'_w2v.csv')